**Nama: Elvi Khairunnisa**

<br>

Import Libraries

In [1]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API of course! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App
2. 2 poitns - Created functionality to read or get specific data from the database
3. 4 points - Created functionality to input new data into each table for the databases
4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [2]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [3]:
trips.tail()

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
17977,23515345,Local365,19476,2021-02-13 14:17:15 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,62
17978,23553979,Local365,19418,2021-02-26 18:12:13 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,4
17979,23499655,U.T. Student Membership,21553,2021-02-07 17:15:29 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,3
17980,23538444,U.T. Student Membership,2147,2021-02-22 20:52:06 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,704
17981,23536417,Local365,16410,2021-02-22 13:41:36 UTC,3838.0,26th/Nueces,3841.0,23rd/Rio Grande,63


**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [4]:
stations

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04T12:00:00Z
1,2500,Republic Square,closed,425 W 4th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,2021-01-04T12:00:00Z
2,2536,Waller & 6th St.,closed,602 Waller St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2021-01-04T12:00:00Z
3,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04T12:00:00Z
4,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021-01-04T12:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,4048,South Congress @ Bouldin Creek,active,799 South Congress Avenue,NaN,32746.0,undetermined_parking,13.0,solar,40.0,5.0,NaN,9,2021-01-04T12:00:00Z
94,3377,MoPac Pedestrian Bridge @ Veterans Drive,active,2105 Veterans Dr.,NaN,16598.0,undetermined_parking,19.0,solar,55.0,5.0,parkland in ROW/eastment,9,2021-01-04T12:00:00Z
95,3794,Dean Keeton & Speedway,active,180 E. Dean Keeton St.,NaN,32670.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [5]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()
conn

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [6]:
# Get the data values
data = tuple(stations.iloc[1].fillna('').values)

# Make the query
query = f"""
INSERT or REPLACE INTO stations
VALUES {data}
"""
data

(2500,
 'Republic Square',
 'closed',
 '425 W 4th Street',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 9,
 '2021-01-04T12:00:00Z')

In [7]:
# See the actual query looks like
print(query)


INSERT or REPLACE INTO stations
VALUES (2500, 'Republic Square', 'closed', '425 W 4th Street', '', '', '', '', '', '', '', '', 9, '2021-01-04T12:00:00Z')



In [8]:
# Execute the query 
conn.execute(query)

In [9]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [10]:
def insert_into_stations(data, conn):
    query = f"""INSERT or REPLACE INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [11]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[6].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [12]:
conn

In [13]:
data

(2546,
 'ACC - West & 12th Street',
 'closed',
 '1231 West Ave.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 9,
 '2021-01-04T12:00:00Z')

In [14]:
result

'OK'

**TASK: Create a Function to insert data for `trips` table**

In [15]:
# Get the data values
data = tuple(trips.iloc[1].fillna('').values)

# Make the query
query = f"""
INSERT or REPLACE INTO trips
VALUES {data}
"""

In [16]:
# See the actual query looks like
print(query)


INSERT or REPLACE INTO trips
VALUES (23459960, 'Local365', '19265', '2021-01-28 08:03:52 UTC', 4054.0, 'Rosewood/Chicon', 4055.0, '11th/Salina ', 16)



In [17]:
# Execute the query 
conn.execute(query)

In [18]:
# Commit changes to database
conn.commit()

In [19]:
def insert_into_trips(data, conn):
    query = f"""INSERT or REPLACE INTO trips values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [20]:
conn = make_connection()
data = tuple(trips.iloc[6].fillna('').values)
result = insert_into_trips(data, conn)

In [21]:
result

'OK'

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [22]:
# Get Specific Station ID Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [23]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2500



In [24]:
# Run the query and get the result
result = pd.read_sql_query(query_station_id, conn)

In [25]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


**Based on specific Station Name**

In [26]:
# Get Specific Station Name Query
name = 'Republic Square'
query_name = f"""
SELECT * FROM stations
WHERE name = {name}
"""

# Get all name Query
query_name_all = "SELECT * FROM stations"

In [27]:
# Run the query and get the result
result = pd.read_sql_query(query_station_id, conn)

In [28]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [29]:
def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

def get_station_name(name, conn):
    query = f"""SELECT * FROM stations WHERE name = {name}"""
    result = pd.read_sql_query(query, conn)
    return result

**TASK: Create a Function to read data from `trips` table**

**Based on specific Trip ID**

In [30]:
# Get Specific Trip ID Query
id = 8270013
query_id = f"""
SELECT * FROM trips
WHERE id = {id}
"""

query_trip_all = "SELECT * FROM trips"

In [31]:
print(query_id)


SELECT * FROM trips
WHERE id = 8270013



In [32]:
result = pd.read_sql_query(query_id, conn)

In [33]:
result

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,8270013,Local365,884,2016-01-01 03:07:08 UTC,2567,Palmer Auditorium,2711,Barton Springs @ Kinney Ave,6


**Based on specific bikeid of Trips**

In [34]:
# Get Specific bikeid of Trips Query
bikeid = 884
query_bikeid = f"""
SELECT * FROM trips
WHERE bikeid = {bikeid}
"""

query_bikeid_all = "SELECT * FROM trips"

In [35]:
print(query_bikeid)


SELECT * FROM trips
WHERE bikeid = 884



In [36]:
result = pd.read_sql_query(query_bikeid, conn)
result

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900293518,Walk Up,884,2015-10-04 21:12:20 UTC,1006,Zilker Park West,1006,Zilker Park West,1
1,9900293981,Local365,884,2015-10-04 21:12:10 UTC,1006,Zilker Park West,2576,Rainey @ River St,31
2,9900296733,Walk Up,884,2015-10-09 19:12:10 UTC,2712,Toomey Rd @ South Lamar,2536,Waller & 6th St.,29
3,9900297658,Local365,884,2015-10-09 16:12:54 UTC,1007,Lavaca & 6th,2712,Toomey Rd @ South Lamar,9
4,9900306591,Local365,884,2015-10-19 11:12:02 UTC,2823,Capital Metro HQ - East 5th at Broadway,2823,Capital Metro HQ - East 5th at Broadway,16
...,...,...,...,...,...,...,...,...,...
2602,23420128,Explorer,884,2021-01-16 15:20:21 UTC,4048,South Congress @ Bouldin Creek,2707,Rainey/Cummings,13
2603,23402510,Local31,884,2021-01-11 08:41:48 UTC,3841,23rd/Rio Grande,3377,Veterans/Atlanta @ MoPac Ped Bridge,19
2604,23374446,Single Trip (Pay-as-you-ride),884,2021-01-02 14:04:12 UTC,2707,Rainey/Cummings,3455,4th/Guadalupe @ Republic Square,10
2605,23424085,24 Hour Walk Up Pass,884,2021-01-17 13:45:32 UTC,2707,Rainey/Cummings,3684,Cesar Chavez/Congress,124


In [37]:
def get_trip_id(id, conn):
    query_id = f"""SELECT * FROM trips WHERE id = {id}"""
    result1 = pd.read_sql_query(query_id, conn)
    return result1 

def get_all_trips(conn):
    query1 = f"""SELECT * FROM trips"""
    result1 = pd.read_sql_query(query1, conn)
    return result1

def get_bikeid(bikeid, conn):
    query_bikeid = f"""SELECT * FROM trips WHERE bikeid = {bikeid}"""
    result2 = pd.read_sql_query(query_bikeid, conn)
    return result2 

# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

```python 
@app.route('/trips/')
def route_all_trips():
    conn = make_connection()
    trips = get_all_trips(conn)
    return trips.to_json()
```

### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

**/stations**

In [38]:
url = 'http://127.0.0.1:5001/stations/'
res = requests.get(url)

In [39]:
res

<Response [200]>

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [40]:
pd.DataFrame(res.json()).head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,,,,,,,,,9,2021-01-04T12:00:00Z
1,2536,Waller & 6th St.,closed,602 Waller St.,,,,,,,,,3,2021-01-04T12:00:00Z
2,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,,,,,,,,,1,2021-01-04T12:00:00Z
3,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,,,,,,,,,1,2021-01-04T12:00:00Z
4,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,,,,,,,,,9,2021-01-04T12:00:00Z


**/trips**

In [42]:
url = 'http://127.0.0.1:5001/trips/'
res = requests.get(url)

In [43]:
res

<Response [200]>

In [44]:
pd.DataFrame(res.json()).head()

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```

**/stations/{station_id}**

Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [45]:
station_id = 2500
url = f"http://127.0.0.1:5001/stations/{station_id}"

In [46]:
url

'http://127.0.0.1:5001/stations/2500'

In [47]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


**/stations_name/'{name}'**

```python
@app.route('/stations_name/<name>')
def route_stations_name(name):
    conn=make_connection()
    name=get_name(name, conn)
    return name.to_json()
```

In [48]:
name = 'Pease Park'
url = f"http://127.0.0.1:5001/stations_name/'{name}'"

In [49]:
url

"http://127.0.0.1:5001/stations_name/'Pease Park'"

In [50]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

**/trips/{id}**

```python
@app.route('/trips/<id>')
def route_trips_id(id):
    conn = make_connection()
    trip = get_trip_id(id, conn)
    return trip.to_json()
```

In [51]:
id = 8270013
url = f"http://127.0.0.1:5001/trips/{id}"

In [52]:
url

'http://127.0.0.1:5001/trips/8270013'

In [53]:
response = requests.get(url)
pd.DataFrame(response.json())

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,8270013,Local365,884,2016-01-01 03:07:08 UTC,2567,Palmer Auditorium,2711,Barton Springs @ Kinney Ave,6


**/trips_bikeid/{bikeid}**

```python
@app.route('/trips/bikeid/<bikeid>')
def route_trips_bikeid(bikeid):
    conn=make_connection()
    bikeid=get_bikeid(bikeid,conn)
    return bikeid.to_json()
```

In [54]:
bikeid = 884
url = f"http://127.0.0.1:5001/trips_bikeid/{bikeid}"

In [55]:
url

'http://127.0.0.1:5001/trips_bikeid/884'

In [56]:
response = requests.get(url)
pd.DataFrame(response.json())

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900293518,Walk Up,884,2015-10-04 21:12:20 UTC,1006,Zilker Park West,1006,Zilker Park West,1
1,9900293981,Local365,884,2015-10-04 21:12:10 UTC,1006,Zilker Park West,2576,Rainey @ River St,31
2,9900296733,Walk Up,884,2015-10-09 19:12:10 UTC,2712,Toomey Rd @ South Lamar,2536,Waller & 6th St.,29
3,9900297658,Local365,884,2015-10-09 16:12:54 UTC,1007,Lavaca & 6th,2712,Toomey Rd @ South Lamar,9
4,9900306591,Local365,884,2015-10-19 11:12:02 UTC,2823,Capital Metro HQ - East 5th at Broadway,2823,Capital Metro HQ - East 5th at Broadway,16
...,...,...,...,...,...,...,...,...,...
2602,23420128,Explorer,884,2021-01-16 15:20:21 UTC,4048,South Congress @ Bouldin Creek,2707,Rainey/Cummings,13
2603,23402510,Local31,884,2021-01-11 08:41:48 UTC,3841,23rd/Rio Grande,3377,Veterans/Atlanta @ MoPac Ped Bridge,19
2604,23374446,Single Trip (Pay-as-you-ride),884,2021-01-02 14:04:12 UTC,2707,Rainey/Cummings,3455,4th/Guadalupe @ Republic Square,10
2605,23424085,24 Hour Walk Up Pass,884,2021-01-17 13:45:32 UTC,2707,Rainey/Cummings,3684,Cesar Chavez/Congress,124


## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

**POST ENDPOINT**

In [57]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "my adress"
}

url = "http://127.0.0.1:5001/json"

In [58]:
res = requests.post(url, json=data)

In [59]:
res

<Response [200]>

In [60]:
res.text

'Hello Andi, your age is 24, and your address in my adress'

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python
@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result
```

In [61]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [62]:
data = stations.iloc[3].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [63]:
data_json

'{"station_id": 2538, "name": "Bullock Museum @ Congress & MLK", "status": "closed", "address": "1881 Congress Ave.", "alternate_name": "", "city_asset_number": "", "property_type": "", "number_of_docks": "", "power_type": "", "footprint_length": "", "footprint_width": "", "notes": "", "council_district": 1, "modified_date": "2021-01-04T12:00:00Z"}'

In [64]:
url = "http://127.0.0.1:5001/stations/add"
res = requests.post(url, json=data_json)

In [65]:
res

<Response [200]>

In [66]:
res.text

'OK'

In [67]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table

for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5001/stations/add"
    res = requests.post(url, json=data_json)

98it [00:08, 11.63it/s]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

```python
@app.route('/trips/add', methods = ['POST']) 
def route_add_trip():
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)

    conn = make_connection()
    result = insert_into_trips(data, conn)
    return result
```

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
data = trips.iloc[10].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [ ]:
data_json

In [ ]:
url = "http://127.0.0.1:5001/trips/add"
res = requests.post(url, json=data_json)

In [ ]:
res

In [ ]:
res.text

In [ ]:
# Sintaks for iteratively insert trips (from trips_csv) into trips table
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5001/trips/add"
    res = requests.post(url, json=data_json)

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint

**/trips/average_duration**

```python
@app.route('/trips/average_duration')
def route_average_duration1():
    conn = make_connection()
    trips = get_average_duration1(conn)
    return trips.to_json()
```

In [68]:
def get_average_duration(conn):
    query=f"""SELECT start_station_name,avg(duration_minutes) as average_duration
    from trips 
    group by start_station_name"""
    result=pd.read_sql_query(query,conn)
    return result

In [69]:
get_average_duration(conn)

,start_station_name,average_duration
0,10th & Red River,46.033708
1,10th/Red River,50.798193
2,11th & Salina,56.659274
3,11th & Salina,60.907135
4,11th & San Jacinto,24.659501
...,...,...
191,West & 6th St.,28.106093
192,Zilker Park,45.559762
193,Zilker Park West,27.168259
194,Zilker Park at Barton Springs & William Barton...,36.292353


**/trips/total_duration**

```python
@app.route('/trips/total_duration')
def route_total_duration():
    conn = make_connection()
    trips = get_total_duration(conn)
    return trips.to_json()
```

In [70]:
def get_total_duration(conn):
    query=f"""SELECT start_station_name,sum(duration_minutes) as total_duration
    from trips 
    group by start_station_name"""
    result=pd.read_sql_query(query,conn)
    return result

In [71]:
get_total_duration(conn)

,start_station_name,total_duration
0,10th & Red River,16388
1,10th/Red River,16865
2,11th & Salina,28103
3,11th & Salina,53781
4,11th & San Jacinto,121596
...,...,...
191,West & 6th St.,410096
192,Zilker Park,1210614
193,Zilker Park West,36976
194,Zilker Park at Barton Springs & William Barton...,418596


**/trips/max_duration**

```python
@app.route('/trips/max_duration')
def route_max_duration():
    conn=make_connection()
    trips=get_max_duration(conn)
    return trips.to_json()
```

In [72]:
def get_max_duration(conn):
    query=f"""SELECT start_station_name,max(duration_minutes) as max_duration
    from trips 
    group by start_station_name"""
    result=pd.read_sql_query(query,conn)
    return result

In [73]:
get_max_duration(conn)

,start_station_name,max_duration
0,10th & Red River,2863
1,10th/Red River,1728
2,11th & Salina,2369
3,11th & Salina,6005
4,11th & San Jacinto,3632
...,...,...
191,West & 6th St.,10981
192,Zilker Park,4036
193,Zilker Park West,1044
194,Zilker Park at Barton Springs & William Barton...,1451


## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint

**/trips/average_duration/{bikeid}**

```python
@app.route('/trips/average_duration/<bikeid>')
def route_average_duration_bikeid(bikeid):
    conn=make_connection()
    trips=get_average_duration_bikeid(bikeid,conn)
    return trips.to_json()
```

In [74]:
trips.dtypes

trip_id                 int64
subscriber_type        object
bikeid                 object
start_time             object
start_station_id      float64
start_station_name     object
end_station_id        float64
end_station_name       object
duration_minutes        int64
dtype: object

In [75]:
trips['bikeid']=trips['bikeid'].str.replace('G','7')
trips['bikeid']=trips['bikeid'].str.replace('B','2')

In [76]:
trips['bikeid'] = trips['bikeid'].astype('int64')

In [77]:
trips.dtypes

trip_id                 int64
subscriber_type        object
bikeid                  int64
start_time             object
start_station_id      float64
start_station_name     object
end_station_id        float64
end_station_name       object
duration_minutes        int64
dtype: object

In [78]:
def get_average_duration_bikeid(bikeid,conn):
    query=f"""SELECT bikeid, avg(duration_minutes) as average_duration
    from trips 
    WHERE bikeid = {bikeid}
    """
    result=pd.read_sql_query(query,conn)
    return result

In [79]:
get_average_duration_bikeid("884",conn)

,bikeid,average_duration
0,884,27.156502


**/trips/total_duration/{bikeid}**

```python
@app.route('/trips/total_duration/<bikeid>')
def route_total_duration_bikeid(bikeid):
    conn=make_connection()
    trips=get_total_duration_bikeid(bikeid,conn)
    return trips.to_json()
```

In [80]:
def get_total_duration_bikeid(bikeid,conn):
    query=f"""SELECT bikeid, sum(duration_minutes) as total_duration
    from trips 
    WHERE bikeid = {bikeid}
    """
    result=pd.read_sql_query(query,conn)
    return result

In [81]:
get_total_duration_bikeid("884",conn)

,bikeid,total_duration
0,884,70797


**/trips/max_duration/{bikeid}**

```python
@app.route('/trips/max_duration/<bikeid>')
def route_max_duration_bikeid(bikeid):
    conn=make_connection()
    trips=get_max_duration_bikeid(bikeid,conn)
    return trips.to_json()
```

In [82]:
def get_max_duration_bikeid(bikeid,conn):
    query=f"""SELECT bikeid, max(duration_minutes) as max_duration
    from trips 
    WHERE bikeid = {bikeid}
    """
    result=pd.read_sql_query(query,conn)
    return result

In [83]:
get_max_duration_bikeid("884",conn)

,bikeid,max_duration
0,884,1942


## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({'"'+specified_date+'%'+'"'})"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

**In Visual Studio Code:**

```python
@app.route('/rent_activities_in_period', methods=['POST'])
def route_rent_activities_in_period():
    input_data = request.get_json(force=True)
    specified_date = input_data['period']

    conn = make_connection()
    query = f"""
    SELECT * FROM trips
    WHERE start_time LIKE ({'"'+specified_date+'%'+'"'})"""
    
    selected_data = pd.read_sql_query(query, conn)
    result = selected_data.groupby('start_station_id').agg({
        'bikeid' : 'count', 
        'duration_minutes' : 'mean'
    })

    return result.to_json()
```

In [84]:
trips.head()

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16
2,23436548,Local365,18331,2021-01-21 07:09:48 UTC,4055.0,11th/Salina,4055.0,11th/Salina,25
3,23382542,Local365,19682,2021-01-04 09:38:18 UTC,4062.0,Lakeshore/Pleasant Valley,3293.0,East 2nd/Pedernales,5
4,23374882,Local365,19437,2021-01-02 15:04:01 UTC,3293.0,East 2nd/Pedernales,4062.0,Lakeshore/Pleasant Valley,5


In [102]:
trips['start_time'] = trips['start_time'].astype('datetime64[ns]')

In [88]:
trips['start_time'] = trips['start_time'].dt.tz_localize(None)

In [89]:
trips.head()

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16
2,23436548,Local365,18331,2021-01-21 07:09:48,4055.0,11th/Salina,4055.0,11th/Salina,25
3,23382542,Local365,19682,2021-01-04 09:38:18,4062.0,Lakeshore/Pleasant Valley,3293.0,East 2nd/Pedernales,5
4,23374882,Local365,19437,2021-01-02 15:04:01,3293.0,East 2nd/Pedernales,4062.0,Lakeshore/Pleasant Valley,5


In [113]:
period = { "period" : "2021-01" }

url = "http://127.0.0.1:5001/rent_activities_in_period"

res = requests.post(url, json=period)

In [114]:
res

<Response [200]>

In [115]:
res.text

'{"bikeid":{"2494":148,"2495":89,"2496":71,"2497":151,"2498":172,"2499":105,"2501":243,"2503":70,"2504":57,"2537":78,"2539":319,"2542":104,"2544":81,"2547":246,"2548":495,"2549":196,"2552":284,"2561":63,"2562":30,"2563":161,"2565":36,"2566":295,"2567":184,"2568":58,"2569":110,"2570":45,"2571":55,"2572":116,"2574":264,"2575":324,"2707":341,"2711":118,"2822":37,"2823":58,"3291":137,"3292":55,"3293":55,"3294":18,"3377":350,"3390":88,"3455":109,"3513":172,"3619":156,"3621":153,"3660":115,"3684":68,"3685":89,"3686":129,"3687":215,"3790":103,"3791":74,"3792":206,"3793":337,"3794":70,"3795":154,"3797":108,"3798":331,"3799":258,"3838":209,"3841":314,"4047":58,"4048":129,"4050":81,"4051":15,"4052":46,"4054":44,"4055":32,"4057":64,"4058":30,"4059":168,"4060":54,"4061":329,"4062":227,"4699":17,"4879":16},"duration_minutes":{"2494":34.5810810811,"2495":118.404494382,"2496":60.1549295775,"2497":27.8013245033,"2498":17.9534883721,"2499":24.4571428571,"2501":37.1522633745,"2503":38.6142857143,"2504":

# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. 
3. Submit your .ipynb and app.py file in your github repository. The dataset is optional to post since it has big size to post. 

